In [1]:
# run model for predict f1 races

#imports 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, log_loss, mean_squared_error

In [2]:
# get data from csv
raw_data = pd.read_csv('../../datasets/f1_data.csv')
raw_data.head(20)

,driver_number,broadcast_name,country_code_x,year,session_name,date_start,time_start,date_end,time_end,gmt_offset,starting_position,wins_before,pit_stops_count,avg_pit_stop_duration,qualifying_position,position_category
0,1,M VERSTAPPEN,NED,2023,Race,2023-03-19,17:00:00,2023-03-19,19:00:00,03:00:00,15,1,0,0.0,1,top3
1,2,L SARGEANT,USA,2023,Race,2023-03-19,17:00:00,2023-03-19,19:00:00,03:00:00,20,0,0,0.0,16,no_points
2,4,L NORRIS,GBR,2023,Race,2023-03-19,17:00:00,2023-03-19,19:00:00,03:00:00,19,0,0,0.0,7,no_points
3,10,P GASLY,FRA,2023,Race,2023-03-19,17:00:00,2023-03-19,19:00:00,03:00:00,9,0,0,0.0,9,points
4,11,S PEREZ,MEX,2023,Race,2023-03-19,17:00:00,2023-03-19,19:00:00,03:00:00,1,0,0,0.0,2,winner
5,14,F ALONSO,ESP,2023,Race,2023-03-19,17:00:00,2023-03-19,19:00:00,03:00:00,2,0,0,0.0,3,top3
6,16,C LECLERC,MON,2023,Race,2023-03-19,17:00:00,2023-03-19,19:00:00,03:00:00,12,0,0,0.0,6,points
7,18,L STROLL,CAN,2023,Race,2023-03-19,17:00:00,2023-03-19,19:00:00,03:00:00,5,0,0,0.0,4,no_points
8,20,K MAGNUSSEN,DEN,2023,Race,2023-03-19,17:00:00,2023-03-19,19:00:00,03:00:00,13,0,0,0.0,17,points
9,21,N DE VRIES,NED,2023,Race,2023-03-19,17:00:00,2023-03-19,19:00:00,03:00:00,18,0,0,0.0,20,no_points


In [3]:
# split into X and y
# data without position category 
X = raw_data.drop(columns=['position_category'])
y = raw_data['position_category']

In [4]:
# wyswitel informacje o zbiorze danych
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

#wyswietl cechy
print("Features in X:", X.columns.tolist())
# wyswietl unikalne kategorie pozycji
print("Unique position categories in y:", y.unique())
print(X.dtypes)

Shape of X: (1635, 15)
Shape of y: (1635,)
Features in X: ['driver_number', 'broadcast_name', 'country_code_x', 'year', 'session_name', 'date_start', 'time_start', 'date_end', 'time_end', 'gmt_offset', 'starting_position', 'wins_before', 'pit_stops_count', 'avg_pit_stop_duration', 'qualifying_position']
Unique position categories in y: ['top3' 'no_points' 'points' 'winner']
driver_number              int64
broadcast_name            object
country_code_x            object
year                       int64
session_name              object
date_start                object
time_start                object
date_end                  object
time_end                  object
gmt_offset                object
starting_position          int64
wins_before                int64
pit_stops_count            int64
avg_pit_stop_duration    float64
qualifying_position        int64
dtype: object


In [5]:
# encode data
from sklearn.preprocessing import LabelEncoder, StandardScaler

for col in X.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])

In [6]:
#scaling data 
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [7]:
#split into train and validation sets

X_train, X_valid, y_train, y_valid = train_test_split(X_scaled, y, test_size=0.3, stratify=y, random_state=42)

In [8]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

In [9]:
# compute specificity   
def compute_specificity(conf_mat):
    specificity = []
    for i in range(conf_mat.shape[0]):
        TP = conf_mat[i, i]
        FP = conf_mat[:, i].sum() - TP
        FN = conf_mat[i, :].sum() - TP
        TN = conf_mat.sum() - (TP + FP + FN)
        spec = TN / (TN + FP) if (TN + FP) > 0 else 0
        specificity.append(spec)
    return specificity

In [10]:
# hyperparameter tuning with Optuna

def objective(trial):
    # hyperparameters to tune
    rf_n_estimators = trial.suggest_int("rf_n_estimators", 50, 150)
    rf_max_depth = trial.suggest_int("rf_max_depth", 3, 15)

    gb_n_estimators = trial.suggest_int("gb_n_estimators", 50, 150)
    gb_learning_rate = trial.suggest_float("gb_learning_rate", 0.01, 0.3)
    gb_max_depth = trial.suggest_int("gb_max_depth", 2, 10)

    svm_C = trial.suggest_float("svm_C", 0.1, 10.0, log=True)
    svm_kernel = trial.suggest_categorical("svm_kernel", ["linear", "rbf"])

    # models to ensemble
    clf_rf = RandomForestClassifier(n_estimators=rf_n_estimators, max_depth=rf_max_depth, random_state=42)
    clf_gb = GradientBoostingClassifier(n_estimators=gb_n_estimators, learning_rate=gb_learning_rate,
                                        max_depth=gb_max_depth, random_state=42)
    clf_svm = SVC(C=svm_C, kernel=svm_kernel, probability=True, random_state=42)

    # ensemble model 
    ensemble = VotingClassifier(
        estimators=[('rf', clf_rf), ('gb', clf_gb), ('svm', clf_svm)],
        voting='soft'
    )

    # encode labels
    le = LabelEncoder()
    y_train_enc = le.fit_transform(y_train)
    y_valid_enc = le.transform(y_valid)

    class_labels = le.classes_
    print("Klasy w macierzy pomyłek:", class_labels)

    # cross-validation setup
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # lists to store metrics for each fold
    acc_list, prec_list, rec_list, f1_list, spec_list = [], [], [], [], []
    mse_list, rmse_list, logloss_list = [], [], []

    # new metrics lists
    prec_macro_list, prec_weighted_list = [], []
    rec_macro_list, rec_weighted_list = [], []
    f1_macro_list, f1_weighted_list = [], []

    # globally summed confusion matrix
    conf_mat_sum = np.zeros((len(np.unique(y_train_enc)), len(np.unique(y_train_enc))), dtype=int)

    # cross-validation loop
    for train_index, test_index in kf.split(X_train, y_train_enc):
        X_tr, X_te = X_train[train_index], X_train[test_index]
        y_tr, y_te = y_train_enc[train_index], y_train_enc[test_index]

        ensemble.fit(X_tr, y_tr)
        y_pred_cv = ensemble.predict(X_te)
        y_proba_cv = ensemble.predict_proba(X_te)

        conf_mat_cv = confusion_matrix(y_te, y_pred_cv)
        conf_mat_sum += conf_mat_cv  # each fold summed to global confusion matrix

        acc_list.append(accuracy_score(y_te, y_pred_cv))
        prec_list.append(precision_score(y_te, y_pred_cv, average="micro", zero_division=0))
        rec_list.append(recall_score(y_te, y_pred_cv, average="micro", zero_division=0))
        f1_list.append(f1_score(y_te, y_pred_cv, average="micro", zero_division=0))

        # new metrics calculations
        prec_macro_list.append(precision_score(y_te, y_pred_cv, average="macro", zero_division=0))
        prec_weighted_list.append(precision_score(y_te, y_pred_cv, average="weighted", zero_division=0))

        rec_macro_list.append(recall_score(y_te, y_pred_cv, average="macro", zero_division=0))
        rec_weighted_list.append(recall_score(y_te, y_pred_cv, average="weighted", zero_division=0))

        f1_macro_list.append(f1_score(y_te, y_pred_cv, average="macro", zero_division=0))
        f1_weighted_list.append(f1_score(y_te, y_pred_cv, average="weighted", zero_division=0))

        spec_cv = np.mean(compute_specificity(conf_mat_cv))
        spec_list.append(spec_cv)

        mse_list.append(mean_squared_error(y_te, y_pred_cv))
        rmse_list.append(np.sqrt(mse_list[-1]))
        logloss_list.append(log_loss(y_te, y_proba_cv))

    # average and std of metrics across folds
    acc, acc_std = np.mean(acc_list), np.std(acc_list, ddof=0)
    prec, prec_std = np.mean(prec_list), np.std(prec_list, ddof=0)
    rec, rec_std = np.mean(rec_list), np.std(rec_list, ddof=0)
    f1, f1_std = np.mean(f1_list), np.std(f1_list, ddof=0)
    spec, spec_std = np.mean(spec_list), np.std(spec_list, ddof=0)

    mse, mse_std = np.mean(mse_list), np.std(mse_list, ddof=0)
    rmse, rmse_std = np.mean(rmse_list), np.std(rmse_list, ddof=0)
    logloss, logloss_std = np.mean(logloss_list), np.std(logloss_list, ddof=0)

    # new metrics averages and stds
    prec_macro, prec_macro_std = np.mean(prec_macro_list), np.std(prec_macro_list, ddof=0)
    prec_weighted, prec_weighted_std = np.mean(prec_weighted_list), np.std(prec_weighted_list, ddof=0)

    rec_macro, rec_macro_std = np.mean(rec_macro_list), np.std(rec_macro_list, ddof=0)
    rec_weighted, rec_weighted_std = np.mean(rec_weighted_list), np.std(rec_weighted_list, ddof=0)

    f1_macro, f1_macro_std = np.mean(f1_macro_list), np.std(f1_macro_list, ddof=0)
    f1_weighted, f1_weighted_std = np.mean(f1_weighted_list), np.std(f1_weighted_list, ddof=0)

    # set metrics to return 
    trial.set_user_attr("accuracy", acc)
    trial.set_user_attr("accuracy_std", acc_std)
    trial.set_user_attr("precision", prec)
    trial.set_user_attr("precision_std", prec_std)
    trial.set_user_attr("recall", rec)
    trial.set_user_attr("recall_std", rec_std)
    trial.set_user_attr("f1_score", f1)
    trial.set_user_attr("f1_std", f1_std)
    trial.set_user_attr("specificity_avg", spec)
    trial.set_user_attr("specificity_std", spec_std)

    trial.set_user_attr("mse", mse)
    trial.set_user_attr("mse_std", mse_std)
    trial.set_user_attr("rmse", rmse)
    trial.set_user_attr("rmse_std", rmse_std)
    trial.set_user_attr("logloss", logloss)
    trial.set_user_attr("logloss_std", logloss_std)

    # set new metrics to return
    trial.set_user_attr("precision_macro", prec_macro)
    trial.set_user_attr("precision_macro_std", prec_macro_std)
    trial.set_user_attr("precision_weighted", prec_weighted)
    trial.set_user_attr("precision_weighted_std", prec_weighted_std)

    trial.set_user_attr("recall_macro", rec_macro)
    trial.set_user_attr("recall_macro_std", rec_macro_std)
    trial.set_user_attr("recall_weighted", rec_weighted)
    trial.set_user_attr("recall_weighted_std", rec_weighted_std)

    trial.set_user_attr("f1_macro", f1_macro)
    trial.set_user_attr("f1_macro_std", f1_macro_std)
    trial.set_user_attr("f1_weighted", f1_weighted)
    trial.set_user_attr("f1_weighted_std", f1_weighted_std)

    # save confusion matrix
    trial.set_user_attr("confusion_matrix", conf_mat_sum.tolist())

    # return metric to optimize
    return f1  # or acc if you are optimizing accuracy


In [11]:
import optuna

# run the optimization
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

c:\Users\Sylwia\miniconda3\envs\mt_python_3_13_2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-09-21 22:36:01,850] A new study created in memory with name: no-name-25668c09-bfd0-491f-8afe-4cbb7139ccef


Klasy w macierzy pomyłek: ['no_points' 'points' 'top3' 'winner']


[I 2025-09-21 22:36:06,107] Trial 0 finished with value: 0.7691948211139202 and parameters: {'rf_n_estimators': 122, 'rf_max_depth': 15, 'gb_n_estimators': 67, 'gb_learning_rate': 0.03954693916454726, 'gb_max_depth': 4, 'svm_C': 2.313609966033525, 'svm_kernel': 'rbf'}. Best is trial 0 with value: 0.7691948211139202.


Klasy w macierzy pomyłek: ['no_points' 'points' 'top3' 'winner']


[I 2025-09-21 22:36:12,575] Trial 1 finished with value: 0.8015475369646824 and parameters: {'rf_n_estimators': 136, 'rf_max_depth': 13, 'gb_n_estimators': 68, 'gb_learning_rate': 0.19175708504361783, 'gb_max_depth': 6, 'svm_C': 7.147781815466811, 'svm_kernel': 'rbf'}. Best is trial 1 with value: 0.8015475369646824.


...

[I 2025-09-21 22:45:15,595] Trial 47 finished with value: 0.8111583544012871 and parameters: {'rf_n_estimators': 80, 'rf_max_depth': 15, 'gb_n_estimators': 136, 'gb_learning_rate': 0.2782248686081048, 'gb_max_depth': 8, 'svm_C': 2.333356919060961, 'svm_kernel': 'rbf'}. Best is trial 43 with value: 0.8199034704665594.


Klasy w macierzy pomyłek: ['no_points' 'points' 'top3' 'winner']


[I 2025-09-21 22:45:28,303] Trial 48 finished with value: 0.8181567455757296 and parameters: {'rf_n_estimators': 110, 'rf_max_depth': 14, 'gb_n_estimators': 116, 'gb_learning_rate': 0.25417107972995484, 'gb_max_depth': 9, 'svm_C': 3.4798207554818212, 'svm_kernel': 'linear'}. Best is trial 43 with value: 0.8199034704665594.


Klasy w macierzy pomyłek: ['no_points' 'points' 'top3' 'winner']


[I 2025-09-21 22:45:40,458] Trial 49 finished with value: 0.8076649046196277 and parameters: {'rf_n_estimators': 111, 'rf_max_depth': 11, 'gb_n_estimators': 116, 'gb_learning_rate': 0.25582054693180734, 'gb_max_depth': 8, 'svm_C': 3.3387834271544357, 'svm_kernel': 'linear'}. Best is trial 43 with value: 0.8199034704665594.


In [12]:
# get the best hyperparameters
best_params = study.best_trial.params

# show best hyperparameters
print("🎯 Najlepsze hiperparametry:")
for param, value in best_params.items():
    print(f"{param}: {value}")

🎯 Najlepsze hiperparametry:
rf_n_estimators: 128
rf_max_depth: 14
gb_n_estimators: 77
gb_learning_rate: 0.2560481189664057
gb_max_depth: 10
svm_C: 0.1706824675188329
svm_kernel: rbf


In [13]:
best_trial = study.best_trial


print("\n📊 Metryki najlepszej próby:")

print("--- Mikro ---")
print(f"F1-score (micro): {best_trial.user_attrs['f1_score']:.4f} ± {best_trial.user_attrs['f1_std']:.4f}")
print(f"Accuracy: {best_trial.user_attrs['accuracy']:.4f} ± {best_trial.user_attrs['accuracy_std']:.4f}")
print(f"Precision (micro): {best_trial.user_attrs['precision']:.4f} ± {best_trial.user_attrs['precision_std']:.4f}")
print(f"Recall / Sensitivity (micro): {best_trial.user_attrs['recall']:.4f} ± {best_trial.user_attrs['recall_std']:.4f}")

print("\n--- Makro ---")
print(f"F1-score (macro): {best_trial.user_attrs['f1_macro']:.4f} ± {best_trial.user_attrs['f1_macro_std']:.4f}")
print(f"Precision (macro): {best_trial.user_attrs['precision_macro']:.4f} ± {best_trial.user_attrs['precision_macro_std']:.4f}")
print(f"Recall / Sensitivity (macro): {best_trial.user_attrs['recall_macro']:.4f} ± {best_trial.user_attrs['recall_macro_std']:.4f}")

print("\n--- Weighted ---")
print(f"F1-score (weighted): {best_trial.user_attrs['f1_weighted']:.4f} ± {best_trial.user_attrs['f1_weighted_std']:.4f}")
print(f"Precision (weighted): {best_trial.user_attrs['precision_weighted']:.4f} ± {best_trial.user_attrs['precision_weighted_std']:.4f}")
print(f"Recall / Sensitivity (weighted): {best_trial.user_attrs['recall_weighted']:.4f} ± {best_trial.user_attrs['recall_weighted_std']:.4f}")

print("\n--- Pozostałe ---")
print(f"Specificity (avg): {best_trial.user_attrs['specificity_avg']:.4f} ± {best_trial.user_attrs['specificity_std']:.4f}")
print(f"MSE: {best_trial.user_attrs['mse']:.4f} ± {best_trial.user_attrs['mse_std']:.4f}")
print(f"RMSE: {best_trial.user_attrs['rmse']:.4f} ± {best_trial.user_attrs['rmse_std']:.4f}")
print(f"LogLoss: {best_trial.user_attrs['logloss']:.4f} ± {best_trial.user_attrs['logloss_std']:.4f}")

# metrics for raport master thesis (macro, confusion matrix)
print("\n📊 Do raportu ---")
print(f"F1-score (macro): {best_trial.user_attrs['f1_macro']:.4f} ± {best_trial.user_attrs['f1_macro_std']:.4f}")
print(f"Precision (macro): {best_trial.user_attrs['precision_macro']:.4f} ± {best_trial.user_attrs['precision_macro_std']:.4f}")
print(f"Recall / Sensitivity (macro): {best_trial.user_attrs['recall_macro']:.4f} ± {best_trial.user_attrs['recall_macro_std']:.4f}")
print(f"Specificity (avg): {best_trial.user_attrs['specificity_avg']:.4f} ± {best_trial.user_attrs['specificity_std']:.4f}")
print(f"Accuracy: {best_trial.user_attrs['accuracy']:.4f} ± {best_trial.user_attrs['accuracy_std']:.4f}")

print("\n🧮 Confusion Matrix:")
print(np.array(best_trial.user_attrs["confusion_matrix"]))


📊 Metryki najlepszej próby:
--- Mikro ---
F1-score (micro): 0.8199 ± 0.0266
Accuracy: 0.8199 ± 0.0266
Precision (micro): 0.8199 ± 0.0266
Recall / Sensitivity (micro): 0.8199 ± 0.0266

--- Makro ---
F1-score (macro): 0.7691 ± 0.0299
Precision (macro): 0.7949 ± 0.0410
Recall / Sensitivity (macro): 0.7515 ± 0.0231

--- Weighted ---
F1-score (weighted): 0.8191 ± 0.0263
Precision (weighted): 0.8216 ± 0.0258
Recall / Sensitivity (weighted): 0.8199 ± 0.0266

--- Pozostałe ---
Specificity (avg): 0.9279 ± 0.0105
MSE: 0.2465 ± 0.0338
RMSE: 0.4952 ± 0.0355
LogLoss: 0.5306 ± 0.0512

📊 Do raportu ---
F1-score (macro): 0.7691 ± 0.0299
Precision (macro): 0.7949 ± 0.0410
Recall / Sensitivity (macro): 0.7515 ± 0.0231
Specificity (avg): 0.9279 ± 0.0105
Accuracy: 0.8199 ± 0.0266

🧮 Confusion Matrix:
[[505  61   3   1]
 [ 59 322  17   4]
 [  5  34  71   5]
 [  1   8   8  40]]


In [14]:
print(optuna.__version__)

4.5.0
